### <u> Generate the Planetary Orbit sonifications used in the "_Audible Universe_" planetarium show </u>

**First, import relevant modules:**

In [ ]:
%reload_ext autoreload 
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import ffmpeg as ff
import wavio as wav
from strauss.sonification import Sonification
from strauss.sources import SingleObject
from strauss import channels
from strauss.score import Score
import numpy as np
from strauss.generator import Synthesizer
import IPython.display as ipd
import os
from scipy.interpolate import interp1d

**Collate the notes we are using to represent each planet and their orbital periods, as well as the length of each sonification**

Then, combine these into dictionaries so they can be easily indexed

In [ ]:
datafile = "../data/datasets/landfrac.txt"
data = np.genfromtxt(datafile)

longitude = data[:,0]
waterfrac = 1-data[:,1]

startlong = 180-(96 + 15./60 + 2.2/3600)
# we travel backwards in longitude per the earth's rotation
longgrid = (np.linspace(startlong,720+startlong,2599)%360 - 180.)[::-1] 
wfrac = interp1d(longitude, waterfrac)

wfracgrid = wfrac(longgrid)*0.75 + 0.15
timegrid = np.linspace(0,1,wfracgrid.size)

plt.plot(timegrid, wfracgrid)
plt.show()


In [ ]:
# chord representing the earth (a Gbsus7 chord)
notes = [['B4']] #[['Gb3', 'Db4', 'E4', 'B5']]

# specify audio system (e.g. mono, stereo, 5.1, ...)
system = "stereo"

length = 60.

# set up synth and turn on LP filter
generator = Synthesizer()
generator.modify_preset({'filter':'on'}) 
print(generator.preset)

**Render sonification for specified planet...**

In [ ]:
score =  Score(notes, length)

# volume swell is directly ahead
data = {'cutoff':wfracgrid,
        'time_evo':timegrid,
        'pitch':np.ones(timegrid.size)}

# set up source
events = SingleObject(data.keys())
events.fromdict(data)
events.apply_mapping_functions()

soni = Sonification(score, events, generator, system)
soni.render()

**Listen to and plot the waveforms from the sonification:**

In [ ]:
soni.notebook_display()

**Combine and save sonification to a multi-channel wav** 

NOTE: Change `"../../FILENAME.wav"` to your filepath of choice

In [ ]:
soni.save_combined("../../FILENAME.wav", True)